In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("/kaggle/input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import cv2, re, random
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, models, optimizers


In [ ]:
img_width, img_height = 150, 150
TRAIN_DIR = '/kaggle/input/train/'
TEST_DIR = '/kaggle/input/test/'
train_images_dogs_cats = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]
test_images_dogs_cats = [TEST_DIR+i for i in os.listdir(TEST_DIR)]

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text)]

def contains_cat(text):
    return 'cat' in text

def contains_dog(text):
    return 'dog' in text

In [ ]:
natural_keys(train_images_dogs_cats[7])

In [ ]:
train_images_dogs_cats.sort(key=natural_keys)
train_images_cats = list(filter(lambda path: 'cat' in path, train_images_dogs_cats))
train_images_dogs = list(filter(lambda path: 'dog' in path, train_images_dogs_cats))

In [ ]:
def prepare_data(list_of_images):
    x = []
    y = []
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (img_width, img_height), interpolation=cv2.INTER_CUBIC))
    for i in list_of_images:
        if 'cat' in i:
            y.append(0)
        elif 'dog' in i:
            y.append(1)
    
    return x, y

In [ ]:
X, Y = prepare_data(train_images_dogs_cats)


In [ ]:
print(keras.backend.image_data_format())

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=1)

In [ ]:
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 16

In [ ]:
model = keras.Sequential()

In [ ]:
model.add(layers.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D())

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid')) # Softmax?

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
train_generator = train_datagen.flow(np.array(X_train), Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(np.array(X_val), Y_val, batch_size=batch_size)

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=3,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [ ]:
model.save_weights('model_weights.h5')
model.save('model_keras.h5')

In [ ]:
X_test, Y_test = prepare_data(test_images_dogs_cats)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = val_datagen.flow(np.array(X_test), batch_size=batch_size)
prediction_probabilities = model.predict_generator(test_generator, verbose=1)

In [ ]:
counter = range(1, len(test_images_dogs_cats) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
cols = ['label']

for col in cols:
    solution[col] = solution[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)
    
solution.to_csv("dogsVScats.csv", index=False)

In [ ]:
# from keras.models import load_model
# new_model = load_model('/kaggle/working/model_keras.h5')
# solution.to_csv(path_or_buf="dogsVScats.csv", index=False)